In [22]:
from scipy.signal import butter, filtfilt
import numpy as np
from pydub import AudioSegment

# Hàm để thiết kế bộ lọc thông thấp sử dụng Butterworth bậc cao
def low_pass_filter(data, cutoff_freq, sample_rate, order=10):
    nyquist_rate = sample_rate / 2.0
    normal_cutoff = cutoff_freq / nyquist_rate
    
    # Thiết kế bộ lọc Butterworth bậc cao
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # Sử dụng filtfilt để áp dụng bộ lọc hai chiều (zero-phase filtering)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Đọc file âm thanh sử dụng pydub
audio = AudioSegment.from_file("/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/output_audio_filtered_butterworth.wav")
sample_rate = audio.frame_rate
samples = np.array(audio.get_array_of_samples())

# Áp dụng bộ lọc thông thấp với tần số cắt là 3200 Hz
cutoff_frequency = 3200  # Tần số cắt là 3200 Hz
filtered_samples = low_pass_filter(samples, cutoff_frequency, sample_rate, order=10)

# Đảm bảo giá trị của tín hiệu nằm trong khoảng int16
filtered_samples = np.clip(filtered_samples, -32768, 32767)

# Ghi file âm thanh đã xử lý
filtered_samples = filtered_samples.astype(np.int16)
output_audio = audio._spawn(filtered_samples.tobytes())
output_audio.export("output_audio_filtered_butterworth.wav", format="wav")


<_io.BufferedRandom name='output_audio_filtered_butterworth.wav'>

# test

In [27]:
import numpy as np
from scipy.fft import fft, ifft
from pydub import AudioSegment
import random

# Hàm để cắt ngẫu nhiên các đoạn tần số khác nhau với các khoảng thời gian khác nhau
def zero_out_random_frequencies_in_segments(audio_path, output_path, num_segments, max_duration, freq_ranges):
    # Đọc file âm thanh
    audio = AudioSegment.from_file(audio_path)
    samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
    sample_rate = audio.frame_rate
    n_samples = len(samples)

    # Tạo danh sách các khoảng thời gian ngẫu nhiên
    random_segments = []
    for _ in range(num_segments):
        # Chọn độ dài ngẫu nhiên cho khoảng thời gian
        duration = random.randint(1, max_duration) * sample_rate  # Giây
        start_time = random.randint(0, n_samples - duration)
        end_time = start_time + duration
        random_segments.append((start_time, end_time))

    # Áp dụng cho từng khoảng thời gian ngẫu nhiên
    for start_time, end_time in random_segments:
        # Tách đoạn tín hiệu cần xử lý
        segment_to_process = samples[start_time:end_time]

        # Thực hiện biến đổi Fourier
        fft_data = fft(segment_to_process)

        # Chọn ngẫu nhiên dải tần số để làm biên độ bằng 0
        low_freq, high_freq = random.choice(freq_ranges)

        # Tính toán các chỉ số tương ứng với dải tần số
        n = len(segment_to_process)
        freqs = np.fft.fftfreq(n, d=1/sample_rate)
        low_idx = np.abs(freqs - low_freq).argmin()
        high_idx = np.abs(freqs - high_freq).argmin()

        # Làm bằng 0 biên độ trong dải tần số mong muốn
        fft_data[low_idx:high_idx] = 0
        fft_data[-high_idx:-low_idx] = 0  # Đối xứng

        # Chuyển đổi ngược sang miền thời gian
        filtered_segment = np.real(ifft(fft_data))

        # Đảm bảo giá trị của tín hiệu nằm trong khoảng int16
        filtered_segment = np.clip(filtered_segment, -32768, 32767)

        # Thay thế đoạn tín hiệu ban đầu bằng đoạn đã lọc
        samples[start_time:end_time] = filtered_segment

    # Chuyển đổi tín hiệu trở lại dạng int16
    samples = samples.astype(np.int16)

    # Ghi file âm thanh đã xử lý
    output_audio = audio._spawn(samples.tobytes())
    output_audio.export(output_path, format="wav")

# Tham số
audio_path = "/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/output_audio_filtered_butterworth.wav"
output_path = "output_audio_random_freq_segments.wav"
num_segments = 5  # Số lượng khoảng thời gian ngẫu nhiên
max_duration = 3  # Độ dài tối đa mỗi khoảng thời gian (giây)
freq_ranges = [(1000, 2000), (0, 1000), (2000, 3000), (800, 1600)]  # Các dải tần số để làm mất biên độ

# Gọi hàm
zero_out_random_frequencies_in_segments(audio_path, output_path, num_segments, max_duration, freq_ranges)


In [33]:
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.fft import fft, ifft
from pydub import AudioSegment
import random

# Hàm để thiết kế bộ lọc thông thấp sử dụng Butterworth bậc cao
def low_pass_filter(data, cutoff_freq, sample_rate, order=10):
    nyquist_rate = sample_rate / 2.0
    normal_cutoff = cutoff_freq / nyquist_rate
    
    # Thiết kế bộ lọc Butterworth bậc cao
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # Sử dụng filtfilt để áp dụng bộ lọc hai chiều (zero-phase filtering)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Hàm để cắt ngẫu nhiên các đoạn tần số với các khoảng thời gian khác nhau
def zero_out_random_frequency_in_segments(samples, sample_rate, num_segments, max_duration, freq_ranges):
    n_samples = len(samples)

    # Tạo danh sách các khoảng thời gian ngẫu nhiên
    for _ in range(num_segments):
        # Chọn độ dài ngẫu nhiên cho khoảng thời gian
        duration = random.randint(1, max_duration) * sample_rate  # Giây
        start_time = random.randint(0, n_samples - duration)
        end_time = start_time + duration
        
        # Tách đoạn tín hiệu cần xử lý
        segment_to_process = samples[start_time:end_time]

        # Thực hiện biến đổi Fourier
        fft_data = fft(segment_to_process)

        # Chọn ngẫu nhiên dải tần số để làm biên độ bằng 0
        low_freq, high_freq = random.choice(freq_ranges)

        # Tính toán các chỉ số tương ứng với dải tần số
        n = len(segment_to_process)
        freqs = np.fft.fftfreq(n, d=1/sample_rate)
        low_idx = np.abs(freqs - low_freq).argmin()
        high_idx = np.abs(freqs - high_freq).argmin()

        # Làm bằng 0 biên độ trong dải tần số mong muốn
        fft_data[low_idx:high_idx] = 0
        fft_data[-high_idx:-low_idx] = 0  # Đối xứng

        # Chuyển đổi ngược sang miền thời gian
        filtered_segment = np.real(ifft(fft_data))

        # Đảm bảo giá trị của tín hiệu nằm trong khoảng int16
        filtered_segment = np.clip(filtered_segment, -32768, 32767)

        # Thay thế đoạn tín hiệu ban đầu bằng đoạn đã lọc
        samples[start_time:end_time] = filtered_segment

    return samples

# Đường dẫn file âm thanh
audio_path = "/home/pdnguyen/fast_confomer_finetun/finetune-fast-conformer/44.686_57.234_spk0.wav"
output_path = "output_audio_filtered_and_random_segments.wav"

# Đọc file âm thanh
audio = AudioSegment.from_file(audio_path)
sample_rate = audio.frame_rate
samples = np.array(audio.get_array_of_samples(), dtype=np.float32)

# Áp dụng bộ lọc thông thấp với tần số cắt là 3200 Hz
cutoff_frequency = 3200  # Tần số cắt là 3200 Hz
filtered_samples = low_pass_filter(samples, cutoff_frequency, sample_rate, order=10)

# Đảm bảo giá trị của tín hiệu nằm trong khoảng int16
filtered_samples = np.clip(filtered_samples, -32768, 32767)

# Cắt ngẫu nhiên các dải tần số trong âm thanh đã lọc
num_segments = 10  # Số lượng khoảng thời gian ngẫu nhiên
max_duration = 3  # Độ dài tối đa mỗi khoảng thời gian (giây)
freq_ranges = [(1000, 2000), (0, 1000), (2000, 3000), (800, 1600)]  # Các dải tần số để làm mất biên độ

# Gọi hàm để làm mất biên độ trong các khoảng ngẫu nhiên
final_samples = zero_out_random_frequency_in_segments(filtered_samples, sample_rate, num_segments, max_duration, freq_ranges)

# Chuyển đổi tín hiệu trở lại dạng int16
final_samples = final_samples.astype(np.int16)

# Ghi file âm thanh đã xử lý
output_audio = audio._spawn(final_samples.tobytes())
output_audio.export(output_path, format="wav")


<_io.BufferedRandom name='output_audio_filtered_and_random_segments.wav'>